In [0]:
import keras
from keras.utils import plot_model
import numpy as np 
import pandas as pd
from PIL import Image, ImageFilter
import os, random
from keras.models import Sequential
from keras.layers import    Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
import matplotlib.pyplot as plt 
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

TRAIN_DIR1 = '/content/drive/My Drive/Colab Notebooks/soldiers/train/'
TEST_DIR1 = '/content/drive/My Drive/Colab Notebooks/soldiers/test/'
TRAIN_DIR2 = '/content/drive/My Drive/Colab Notebooks/civil/train/'
TEST_DIR2 = '/content/drive/My Drive/Colab Notebooks/civil/test/'

ROWS = 224
COLS = 224

CHANNELS = 3
#print(os.listdir(TRAIN_DIR))
#print()
#print(os.listdir(TEST_DIR))
#print()
train_images1 = [TRAIN_DIR1 + i for i in os.listdir(TRAIN_DIR1)]
test_images1 = [TEST_DIR1 + i for i in os.listdir(TEST_DIR1)]
train_images2 = [TRAIN_DIR2 + i for i in os.listdir(TRAIN_DIR2)]
test_images2 = [TEST_DIR2 + i for i in os.listdir(TEST_DIR2)]
#print(train_images)
#print()
#print(test_images)
#print()
def read_image(file_path):
    img = Image.open(file_path)
    im=img.convert("L")
    return im.resize((ROWS, COLS), Image.BICUBIC)

    
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype = np.uint8)
    
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        image = image.transpose(Image.ROTATE_90)
        data[i]=image.transpose(Image.FLIP_LEFT_RIGHT)
        
        if i % 1 == 0:
            print ('Processed {} of {}'.format(i, count))
            
    return data

train1 = prep_data(train_images1)  
train2 = prep_data(train_images2)
test1 = prep_data(test_images1)  
test2 = prep_data(test_images2)

print("Train shape: {}".format(train.shape))
print("Test shape:  {}".format(test.shape))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
labels = []
for i in train_images1:
    labels.append(1)
for i in train_images2:
    labels.append(0)
        
train = np.concatenate((train1,train2),axis=1)
test = np.concatenate((test1,test2),axis=1)

        
train = train.reshape(-1,224,224,3)
test = test.reshape(-1,224, 224, 3)
X_train = train.astype('float32')
X_test = test.astype('float32')
X_train /= 255.0
X_test /= 255.0
Y_train = labels
X_train=np.array(X_train)
#print(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
print(Y_train)
labels_test = []
for i in test_images1:
    labels_test.append(1)
for i in test_images2:
    labels_test.append(0)
Y_test = labels_test
Y_test = np.array(Y_test)
print(Y_test)

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
vgg16_model = VGG16(weights='imagenet', include_top=False)
# vgg_model = VGG16(weights='imagenet', include_top=False)
# type(vgg_model)

In [0]:
print("Training matrix shape ", X_train.shape) #  m X features
print("Testing matrix shape  ", X_test.shape) #  m X features

In [0]:
features_x=model.predict(X_train)
print(features_x.shape)
X_train = features_x.reshape(322,25088)
print(X_train.shape)

In [0]:
features_xtest=model.predict(X_test)
print(features_xtest.shape)
X_test=features_xtest.reshape(200,25088)
print(X_test.shape)

In [0]:
model=Sequential()
for layer in vgg16_model.layers:
  model.add(layer)
  
for layer in model.layers:
  layer.trainable=False

model.add(Dense(256, input_dim=25088, activation='relu',kernel_initializer='glorot_uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)
model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

model.add(Dense(128,input_dim=256,activation='sigmoid'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)

#model.add(Dense(4096,input_dim=4096,activation='sigmoid'))
#keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)

#model.add(Dense(4096,input_dim=1024,activation='relu'))
#keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)
#sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

model.add(Dense(1,input_dim=128,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# fitting the model 
model.summary()
#model.fit(X_train, Y_train, epochs=30)

In [0]:
from sklearn.metrics import log_loss
pred = model.predict(X_test, batch_size=32, verbose=1)
print(pred)
score = log_loss(Y_test, pred)
print(score)
for i in range(200):
    if pred[i] > 0.5:
        pred[i]=1
    else:
        pred[i]=0
print(pred)
print("Y_test", Y_test)
print("Predicted", np.array(pred))

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,pred)

**Explaination**

In [0]:
!pip install lime
import lime

In [0]:
from lime import lime_image
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance("solider.jpg", model.predict, hide_color=0, num_samples=100)

In [0]:
from skimage.segmentation import mark_boundaries
temp, mask = explanation.get_image_and_mask(295, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))